In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd

In [ ]:
def get_wikipedia_data(url):
    s=Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    driver.maximize_window()
    driver.get(url)
    time.sleep(5)

    # Locate the table and rows
    table = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[3]/main/div[3]/div[3]/div[1]/table[3]")
    rows = table.find_elements(By.XPATH, "/html/body/div[2]/div/div[3]/main/div[3]/div[3]/div[1]/table[3]/tbody/tr[*]")

    data = []

    # Extract data from each row
    for i in range(1, len(rows)):
        tds = rows[i].find_elements(By.TAG_NAME, "td")
        if len(tds) >= 7:
            values = {
                'rank': i,
                'stadium': tds[0].text.strip(),
                'capacity': tds[1].text.strip().replace(',', '').replace('.', ''),
                'region': tds[2].text.strip(),
                'country': tds[3].text.strip(),
                'city': tds[4].text.strip(),
                'images': 'https://' + tds[5].find_element(By.TAG_NAME, 'img').get_attribute('src').split("//")[1] if tds[5].find_elements(By.TAG_NAME, 'img') else 'NO_IMAGE',
                'home_team': tds[6].text.strip(),
            }
            data.append(values)
    return data

def write_data_to_csv(data):
    df = pd.DataFrame(data)
    # Remove the number in brackets
    df['capacity'] = df['capacity'].str.replace(r'\[\d+\]', '', regex=True)
    df['capacity'] = df['capacity'].astype(int)

    df.to_csv(r'C:\Users\omarn\Downloads\stadiums.csv', index=False)
    print('Data written to stadiums.csv')

# URL of the Wikipedia page to scrape
url = "https://en.wikipedia.org/wiki/List_of_association_football_stadiums_by_capacity"
stadium_data = get_wikipedia_data(url)
write_data_to_csv(stadium_data)